In [ ]:
#Import necessary packages and elements
import pandas as pd
import datetime as dt
import numpy as np
import math
import sqlalchemy
import scipy as sp
from scipy import stats
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table, Column, Integer, Float, String
from scipy.stats import multivariate_normal
from scipy.stats import multivariate_normal
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_regression, SelectKBest, chi2, RFE, f_classif
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.externals import joblib

import warnings 
warnings.filterwarnings('ignore')

### Connect to database, truncate table, read data

In [ ]:
#Connect to database
engine = create_engine("(masked)/OASIS_Sandbox?driver=SQL+Server+Native+Client+11.0?(masked)")

In [ ]:
#Rebuild the table to train the model
building_table = ("""
drop table if exists #tempnoshow
drop table if exists #prevbilling
drop table if exists #appttypes
drop table if exists #apptkeys
drop table if exists #diagnoses
drop table if exists 'oasis_sandbox.cat.gbusch_NoShowFeatures'

SELECT 
DP.[MRN Alpha]

,dd.Date
,case when [Disease Type] not in ('Unmapped','No Value') then [Disease Type] else [Disease Subgroup] end 'Disease Type'
into #diagnoses
from oasis.mart.v_FactEnc FE
inner join oasis.mart.v_DimPatient DP on DP.[Pat Key] = fe.[Pat Key]
inner join oasis.mart.v_DImDate dd on dd.[Date Key] = fe.[Discharge Date Key] and dd.Date >= dateadd(day,-910,getdate())
inner join oasis.mart.v_DimDiagnosis dd1 on dd1.[Dx Key] = fe.[Mapped Dx Key]
where [Disease Type] not in ('Unmapped','No Value') or [Disease Subgroup] <> 'Unmapped'

select 
1 as 'visits'
,f.[Appt Cancel Lead Days]
,ds.[Appt Status Name]
,case when [Appt Status Name] in ('Completed','Arrived') then 1 
when [Appt Status Name] = 'No Show' then 0 
when ([Appt Cancel Lead Days] <= 2 and [Appt Cancel Lead Days] >= 0) then 0  end as 'Completed Appt'
,case when ds.[Appt Status Name] = 'No Show' then 1 
when ([Appt Cancel Lead Days] <= 2 and [Appt Cancel Lead Days] >= 0) then 1 
--when ds.[Appt Status Name] = 'Scheduled' then 1 
when ds.[Appt Status Name] in ('Completed','Arrived') then 0 end as 'No Show Appts'
,da.[Contact Serial Number]
,f.[Appt Date Time]
,datediff(day,[Appt Made Date],dd.Date) as 'leadtime'
,dag.Age
,case when dp.Sex = 'Male' then 1 else 0 end as 'ismale'
,dat.[Appt Type Class]
,case when db.[SCCA Contract] is not null then (case when db.[SCCA Contract] like 'Premera%' and db.[Payor Class] not like 'Medi%' then 1 else 0 end) end

 as 'TopPayor'
,case when db.[Payor Class] is not null then
(case when db.[Payor Subclass] like 'Regence%' or 
db.[Payor Subclass] like 'United%' or 
db.[Payor Subclass] like 'BCBS%' or 
db.[Payor Subclass] like 'Aetna%' or 
db.[Payor Subclass] like 'BDCT%' and db.[Payor Class] not like 'Medi%' then 1 else 0 end)  end as 'Next5Payors'
,case when db.[Payor Class] is not null then (case when db.[Payor Class] like 'Medicaid%' then 1 else 0 end) end as 'Medicaid'
,case when db.[Payor Class] is not null then (case when db.[Payor Class] like 'Medicare%' then 1 else 0 end) end as 'Medicare'
,case when db.[Payor Class] is not null then (case when db.[Payor Class] like 'Self Pay' then 1 else 0 end) end as 'SelfPay'

,dd.Date as 'Appt Date'
,row_number() over (partition by f.[Pat Key],dd.[Date Key] order by f.[Appt Date Time] asc) as 'RowNum'
,dp.[MRN Alpha] 'MRN'
, case when [Employment status] in ('On Active Military Duty','Employed Full-time','Self-employed','Employed Part-time') then 1 else 0 end as 'employed'
,dsl.[Service Line Key]
,z.Latitude 'PatLat'
,z.Longitude 'PatLong'
,zc.Latitude 'SiteLat'
,zc.Longitude 'SiteLong'
,case when dat.[Rpt Grp Six] = 'New' then 1 else 0 end as 'newappt'
,case when dat.[Rpt Grp Six] = 'Return' then 1 else 0 end as 'retappt'
, case when [Phone Number] in ('Unknown','No Match') then 1 else 0 end as 'nophonenumber'
,case when race in ('Unknown','Other','No Match') then 1 else 0 end as 'otherrace'
,case when race = 'Multiracial' then 1 else 0 end as 'multirace'
, case when race = 'Black or African American' then 1 else 0 end as 'africanamericanrace'
, case when race = 'American Indian or Alaska Native' then 1 else 0 end as 'americanindianrace'
, case when race = 'Hispanic' then 1 else 0 end as 'hispanicrace'
, case when race = 'White' then 1 else 0 end as 'whiterace'
, case when race = 'Native Hawaiian or Other Pacific Islander' then 1 else 0 end 'pacislanderrace'
, case when race = 'Asian' then 1 else 0 end as 'asianrace'
,f.[Pat Key]
,dd.[Date Key]
,dpp.[Prov Type Subcategory]
,f.[Sched Appt Length]
,dat.[Appt Type Key]
,ddiag.[Disease Type]
,ddiag.[Disease Subgroup]
,dsl.[Service Line]
,[Prov Name]

into #tempnoshow
FROM oasis.mart.v_FactAppt f 
inner join oasis.mart.v_DimApptStatus ds on ds.[Appt Status Key] = f.[Appt Status Key]
inner join oasis.mart.v_DimAppointment da on da.[Appt Key] = f.[Appt Key]
inner join oasis.mart.v_DimSite dss on dss.[Site Key] = f.[Site Key]
inner join oasis.mart.v_DimDate dd on dd.[Date Key] = f.[Appt Date Key]
inner join oasis.mart.v_DimPatient dp on dp.[Pat Key] = f.[Pat Key]
left join oasis.mart.v_FactEncAll fe on fe.[Enc Key] = f.[Enc Key]
left join oasis.mart.v_DimPayorBenPlan db on db.[Payor Ben Plan Key] = fe.[Payor Ben Plan Key]
inner join oasis.mart.v_DimAge dag on dag.[Age Key] = f.[Pat Appt Age Key]
INNER JOIN OASIS_Sandbox.dbo.ZipCode z ON dp.[Zip Code] = z.ZipCode
INNER JOIN OASIS_Sandbox.dbo.ZipCode zc on zc.[Site] = dss.[Site]
INNER JOIN oasis.mart.V_DimServiceLine DSL on DSL.[Service Line Key] = f.[Service Line Key]
inner join oasis.mart.v_DimApptType dat on dat.[Appt Type Key] = f.[Appt Type Key]
inner join oasis.mart.v_DimProvider dpp on dpp.[Prov Key] = f.[Primary Prov Key]
inner join oasis.mart.v_FactPatient FP on FP.[Pat Key] = f.[Pat Key]
inner join oasis.mart.v_DimDiagnosis ddiag on ddiag.[Dx Key] = fp.[Last Enc Mapped Dx Key]

where 1=1
and ([Appt Cancel Lead Days] < 3 or ds.[Appt Status Name] in ('Arrived','Completed','No Show','Scheduled'))
and not (dd.Date <= dateadd(day,-7,getdate()) and ds.[Appt Status Name] = 'Scheduled')
and f.[Death Date] is null
and [Appt Type Name] not like '%phone%'
and dss.[Site] = 'SLU'
and not dat.[Appt Type name] ='PERIPHERAL DRAW UW 3RD FLR LAB'
and not dat.[Appt Type Name] = 'SCCA LAB'
and not dat.[Appt Type Name] = 'PAIN PROTOCOL'
and not dat.[Appt Type Name] = 'FOOD SAFETY CLASS'
and not dat.[Appt Type Name] = 'SOCIAL WORKER VISIT-GO'
and not dat.[Appt Type Name] = 'PAIN-MODIFIED PROTOCOL'
and not dat.[Appt Type Name] = 'SCCA PHYSICAL THERAPY RET 60'
and not dat.[Appt Type Name] like '%peds %'
and not dat.[Appt Type Name] like '%pediatric %'
and not dat.[Appt Type Name] like '%PUMP DISCONNECT%'
--and not dsl.[Service Line] = 'Transplant'


-- First temp table to identify low % appt types by service line
SELECT dat.[Appt Type Name],count(*) 'count'
,sum(case when ds.[Appt Status Name] = 'No Show' then 1 when ([Appt Cancel Lead Days] <= 2 and [Appt Cancel Lead Days] >= 0) 
then 1 when ds.[Appt Status Name] in ('Completed','Arrived') then 0 end) as 'No Show Appts1'
,dat.[Appt Type Key]
,dsl.[Service Line Key]
into #appttypes
 FROM oasis.mart.v_FactAppt fa
inner join OASIS.mart.v_DimApptType dat on dat.[Appt Type Key] = fa.[Appt Type Key]
inner join oasis.mart.v_DimApptStatus ds on ds.[Appt Status Key] = fa.[Appt Status Key]
inner join oasis.mart.v_DimDate dd on dd.[Date Key] = fa.[Appt Date Key] and date between dateadd(day,-730,getdate()) and dateadd(day,30,GETDATE())
inner join oasis.mart.v_DimServiceLine dsl on dsl.[Service Line Key] = fa.[Service Line Key]

group by dat.[Appt Type Name]
,dat.[Appt Type Key]
,dsl.[Service Line Key]


select *

,left(cast(ISNULL([No Show Appts1] * 100,0) as decimal) / cast(([Count] * 100) as decimal),4) 'Percent'
into #apptkeys
 from #appttypes

where ([Count] >20
or cast(left(cast(ISNULL([No Show Appts1] * 100,0) as decimal) / cast(([Count] * 100) as decimal),4) as decimal) <.60)
and not cast(left(cast(ISNULL([No Show Appts1] * 100,0) as decimal) / cast(([Count] * 100) as decimal),4) as decimal) >=.80



select
db1.[SCCA Contract]
,db1.[Payor Class]
,db1.[Payor Subclass]
,dp1.[MRN Alpha]
,dd1.Date

into #prevbilling
from oasis.mart.v_FactEncAll fe1 
inner join oasis.mart.v_DimPayorBenPlan db1 on db1.[Payor Ben Plan Key] = fe1.[Payor Ben Plan Key]
inner join oasis.mart.v_DimDate dd1 on dd1.[Date Key] = fe1.[Discharge Date Key]
inner join oasis.mart.v_DimPatient dp1 on dp1.[Pat Key] = fe1.[Pat Key]







select 
MRN 'MRN'
,(select count(*)

from #tempnoshow p
where p.[MRN] = a.MRN and cast(p.[Appt Date Time] as date) = cast(a.[Appt Date Time] as date) ) 'totalapptstoday'
,[Contact Serial Number] 'Contact Serial Number'
,[Appt Date Time] 'Appt Date Time'
,ISNULL([No Show Appts],0) 'No show'
,[Sched Appt Length] as 'length'
,(SELECT sum(1)
from #tempnoshow b 
where a.[Pat Key] = b.[Pat Key] and a.[Date Key] >= b.[Date Key]
group by b.[Pat Key]
) as 'NumAppt'
,COALESCE((SELECT sum(b.[No show Appts])
from #tempnoshow b 
where a.[Pat Key] = b.[Pat Key] and a.[Date Key] > b.[Date Key]
group by b.[Pat Key]
),0) as 'NumAptNS'
, case when datepart(dw,[Appt Date]) =2 then 1 else 0 end as 'ApptMon'
, case when datepart(dw,[Appt Date]) =3 then 1 else 0 end as 'ApptTue'
, case when datepart(dw,[Appt Date]) =4 then 1 else 0 end as 'ApptWed'
, case when datepart(dw,[Appt Date]) =5 then 1 else 0 end as 'ApptThur'
, case when datepart(dw,[Appt Date]) =6 then 1 else 0 end as 'ApptFri'
, case when datepart(hh,[Appt Date Time]) between 4 and 11 then 1 else 0 end as 'AMappt'
,ISNULL(LAG([No show Appts],1) OVER (partition by [Pat Key] ORDER BY [Date Key] asc),0) as 'prevnoshow'
, case when a.[Age] between 18 and 35 then 1 else 0 end as 'Age18to35'
, case when a.[Age] between 36 and 55 then 1 else 0 end as 'Age36to55'
, case when a.[Age] between 56 and 75 then 1 else 0 end as 'Age56to75'
, case when a.[Age] > 75 then 1 else 0 end as 'Age76'
,case when abs(ISNULL([leadtime],0))= 0 then .1 else abs(ISNULL([leadtime],0)) end 'leadtime'
,COALESCE(case when [TopPayor] is not null then [TopPayor] else
(SELECT TOP 1 case when p.[SCCA Contract] like 'Premera%' and p.[Payor Class] not like 'Medi%' then 1 else 0 end
from
#prevbilling p
where p.[MRN Alpha] = a.[MRN]
and p.Date<a.[Appt Date]
order by p.Date desc
) end,0)
 as 'TopPayor'
,COALESCE(case when [Next5Payors] is not null then [Next5Payors] else
(SELECT TOP 1 case when p.[Payor Subclass] like 'Regence%' or 
p.[Payor Subclass] like 'United%' or 
p.[Payor Subclass] like 'BCBS%' or 
p.[Payor Subclass] like 'Aetna%' or 
p.[Payor Subclass] like 'BDCT%' and p.[Payor Class] not like 'Medi%' then 1 else 0 end
from
#prevbilling p
where p.[MRN Alpha] = a.[MRN]
and p.Date<a.[Appt Date]
order by p.Date desc
) end,0)
 as 'Next5Payors'
, COALESCE(case when [medicaid] is not null then [Medicaid] else
(SELECT TOP 1 case when p.[Payor Class] like 'Medicaid%' then 1 else 0 end 
from
#prevbilling p
where p.[MRN Alpha] = a.[MRN]
and p.Date<a.[Appt Date]
order by p.Date desc
) end,0)
as 'Medicaid'
,COALESCE(case when [medicare] is not null then [Medicare] else
(SELECT TOP 1 case when p.[Payor Class] is not null and p.[Payor Class] like 'Medicare%' then 1 else 0 end 
from
#prevbilling p
where p.[MRN Alpha] = a.[MRN]
and p.Date<a.[Appt Date]
order by p.Date desc
) end,0) as 'Medicare' 
,COALESCE(case when [selfpay] is not null then [selfpay] else
(SELECT TOP 1 case when p.[Payor Class] is not null and p.[Payor Class] like 'Self Pay' then 1 else 0 end 
from
#prevbilling p
where p.[MRN Alpha] = a.[MRN]
and p.Date<a.[Appt Date]
order by p.Date desc
) end,0)
 as 'SelfPay'
,ISNULL(case when ABS(ROUND(3959 * ACOS(
SIN(RADIANS([PatLat])) * SIN(RADIANS([SiteLat])) +
COS(RADIANS([PatLat])) * COS(RADIANS([SiteLat])) * COS(RADIANS([SiteLong]) - RADIANS([PatLong]))),1)) = 0 then .1 else 
ABS(ROUND(3959 * ACOS(
SIN(RADIANS([PatLat])) * SIN(RADIANS([SiteLat])) +
COS(RADIANS([PatLat])) * COS(RADIANS([SiteLat])) * COS(RADIANS([SiteLong]) - RADIANS([PatLong]))),1)) end ,.1)
as 'DistanceMiLog'
,africanamericanrace
,americanindianrace
,multirace
,hispanicrace
,otherrace
,whiterace
,pacislanderrace
,ismale
,asianrace
,employed
,nophonenumber
,case when [Appt Type Class] = 'Procedure' then 1 else 0 end as 'procedureappt'
,case when [Appt Type Class] = 'Radiation Oncology' then 1 else 0 end as 'radoncappt'
,case when [Appt Type Class] = 'Physical Therapy' then 1 else 0 end as 'ptappt'
,case when [Appt Type Class] = 'Infusion' then 1 else 0 end as 'infappt'
, case when [Appt Type Class] = 'Clinic' then 1 else 0 end as 'clinicappt'
,case when [Appt Type Class] = 'Interventional Radiology' then 1 else 0 end as 'intradappt'
, case when [Appt Type Class] = 'Apheresis' then 1 else 0 end as 'apheresisappt'
,case when [Appt Type Class] = 'Imaging' then 1 else 0 end as 'imagingappt'
 , case when [Prov Type Subcategory] = 'MA' then 1 else 0 end as 'maprov'
 , case when [Prov Type Subcategory] = 'Resident' then 1 else 0 end as 'residentprov'
  , case when [Prov Type Subcategory] = 'Supportive Care' then 1 else 0 end as 'suppcareprov'
   , case when [Prov Type Subcategory] = 'MD' then 1 else 0 end as 'mdprov'
    , case when [Prov Type Subcategory] = 'APP/Fellow' then 1 else 0 end as 'appprov'
,newappt
,retappt
,[Service Line]
,[Appt Type Name]
,[Prov Name]
,isnull((SELECT count(*) from #tempnoshow abc
where abc.[Appt Type Key] = A.[Appt Type Key] and ABC.MRN = A.MRN and A.[Appt Date Time] > ABC.[Appt Date Time]),0) 'PreviousApptsOfType'
,isnull((SELECT sum(ISNULL(abc1.[No Show Appts],0))
from #tempnoshow ABC1
where ABC1.[Appt Type Key] = A.[Appt Type Key] and ABC1.MRN = A.MRN and A.[Appt Date Time] > ABC1.[Appt Date Time]),0) 'PreviousNSOfType'



,coalesce(case when [Disease Type] not in ('Unmapped','No Value') then [Disease Type] 
when [Disease Type] in ('Unmapped','Value') and [Disease Subgroup] = 'Unmapped' then (select top 1 [Disease Type] from #diagnoses d where d.[MRN Alpha] = a.MRN and d.Date <= a.[Appt Date Time] order by d.date desc)
else [Disease Subgroup] end,'No Prior Dx') 'Disease Type'

into oasis_sandbox.cat.gbusch_NoShowFeatures
from #tempnoshow a
inner join #apptkeys ak on a.[Appt Type Key] = ak.[Appt Type Key] and ak.[Service Line Key] = a.[Service Line Key]
where [Appt Date Time] between dateadd(day,-730,getdate()) and dateadd(day,14,GETDATE())

and not ([Appt Date]<= getdate() and [No show Appts] is null)
and  case when [Prov Type Subcategory] = 'Nurse' then 1 else 0 end = 0
order by [Appt Date Time] desc

""")

with engine.connect() as con:
    con.execution_options(autocommit=True).execute(building_table)

In [ ]:
#Query the NoShowData table
with engine.connect() as con:

    data= pd.read_sql_query('select * from cat.gbusch_NoShowFeatures',con)

In [ ]:
#Truncate the NoShowPredictions table
truncate_query = sqlalchemy.text("TRUNCATE TABLE oasis_sandbox.dbo.NoShowPredictions")
with engine.connect() as con:
    con.execution_options(autocommit=True).execute(truncate_query)

#One hot encode Appt Type, Provider, Disease Type; join back to original data

In [ ]:
AT_df = data[['Contact Serial Number', 'Appt Type Name']]

appttype_df = pd.get_dummies(AT_df['Appt Type Name'], prefix='apptType')
AT_df = pd.merge(AT_df, appttype_df, left_index=True, right_index=True)


AT_df.drop('Appt Type Name', axis=1, inplace=True)

AT_df.head()

In [ ]:
DT_df = data[['Contact Serial Number', 'Disease Type']]

diseasetype_df = pd.get_dummies(DT_df['Disease Type'], prefix='disease')
DT_df = pd.merge(DT_df, diseasetype_df, left_index=True, right_index=True)

DT_df.drop('Disease Type', axis=1, inplace=True)
#DT_df.drop('disease_Unmapped', axis=1, inplace=True)
#DT_df.drop('disease_No Value', axis=1, inplace=True)
DT_df.head()
print(DT_df.shape)

In [ ]:
prov_df = data[['Contact Serial Number', 'Prov Name']]

provname_df = pd.get_dummies(prov_df['Prov Name'], prefix='prov')
prov_df = pd.merge(prov_df, provname_df, left_index=True, right_index=True)

prov_df.drop('prov_Unknown', axis=1, inplace=True)
prov_df.drop('Prov Name', axis=1, inplace=True)

prov_df.head()

In [ ]:
final_data = pd.merge(data, AT_df, how='inner', on='Contact Serial Number')
#final_data = pd.merge(final_data, prov_df, how='inner', on='Contact Serial Number')
final_data = pd.merge(final_data, DT_df, how='inner', on='Contact Serial Number')


In [ ]:
datahistoric = final_data[final_data['Appt Date Time'] < str(dt.datetime.now())]
datafuture = final_data[final_data['Appt Date Time'] >= str(dt.datetime.now())]
dropfuture = ['MRN', 'Contact Serial Number', 'Appt Date Time','No show','Service Line','Disease Type','Appt Type Name','Prov Name']

datafuture = datafuture.drop(dropfuture,axis=1)

#datahistoric.head(50)

In [ ]:
scaler = MinMaxScaler()

# fill missing "prevnoshow" values with 0
datahistoric['prevnoshow'].fillna(value=0, inplace=True)

# drop fields not used in training/test
drophistoric = ['MRN', 'Contact Serial Number', 'Appt Date Time','Service Line','Disease Type','Appt Type Name','Prov Name']
datahistoric = datahistoric.drop(drophistoric,axis=1)


# scale appropriately for later weighting of variables

datahistoric['PreviousNSOfType'] = scaler.fit_transform(datahistoric['PreviousNSOfType'].values.reshape(-1,1))
datahistoric['PreviousApptsOfType'] = scaler.fit_transform(datahistoric['PreviousApptsOfType'].values.reshape(-1,1))
datahistoric['leadtime'] = scaler.fit_transform(datahistoric['leadtime'].values.reshape(-1,1))
datahistoric['DistanceMiLog'] = scaler.fit_transform(datahistoric['DistanceMiLog'].values.reshape(-1,1))
datahistoric['NumAppt'] = scaler.fit_transform(datahistoric['NumAppt'].values.reshape(-1,1))
datahistoric['NumAptNS'] = scaler.fit_transform(datahistoric['NumAptNS'].values.reshape(-1,1))
datahistoric['length'] = scaler.fit_transform(datahistoric['length'].values.reshape(-1,1))
datahistoric['totalapptstoday'] = scaler.fit_transform(datahistoric['totalapptstoday'].values.reshape(-1,1))

In [ ]:

# fill missing "No show" and  "prevnoshow" values with 0
datafuture['prevnoshow'].fillna(value=0, inplace=True)

# scale appropriately for later weighting of variables
datafuture['PreviousNSOfType'] = scaler.fit_transform(datafuture['PreviousNSOfType'].values.reshape(-1,1))
datafuture['PreviousApptsOfType'] = scaler.fit_transform(datafuture['PreviousApptsOfType'].values.reshape(-1,1))
datafuture['leadtime'] = scaler.fit_transform(datafuture['leadtime'].values.reshape(-1,1))
datafuture['DistanceMiLog'] = scaler.fit_transform(datafuture['DistanceMiLog'].values.reshape(-1,1))
datafuture['NumAppt'] = scaler.fit_transform(datafuture['NumAppt'].values.reshape(-1,1))
datafuture['NumAptNS'] = scaler.fit_transform(datafuture['NumAptNS'].values.reshape(-1,1))
datafuture['length'] = scaler.fit_transform(datafuture['length'].values.reshape(-1,1))
datafuture['totalapptstoday'] = scaler.fit_transform(datafuture['totalapptstoday'].values.reshape(-1,1))

In [ ]:
#Split past data into test and train
train_frac = .7
split = int(train_frac * len(datahistoric))

datatrain = datahistoric[:split]
data_train_noshow = datatrain[datatrain['No show'] == 1]
data_train_show = datatrain[datatrain['No show'] == 0]
length = int(len(data_train_noshow)*1.6)
data_train_show = data_train_show.sample(n=length,random_state=42)
datatrain = pd.concat([data_train_noshow,data_train_show])
datatrain = datatrain.sample(frac=1.0)

y_train = datatrain['No show']
X_train = datatrain.drop('No show', axis=1)

datatest = datahistoric[split:]
X_test = datatest.drop('No show', axis=1)
y_test = datatest['No show']

print("X_train dimensions: ", X_train.shape)
print("y_train dimensions: ", y_train.shape)
print("-" * 30)
print("X_test dimensions: ", X_test.shape)
print("y_test dimensions: ", y_test.shape)

#X_train.isnull().any()

#X_feat.head()
#y_feat.head()

In [ ]:
f_values = f_classif(X_train,y_train)
significant_count = 0
significant_idx = []
for i in range(f_values[0].shape[0]):
    if f_values[1][i] <1e-3:
        significant_count += 1
        significant_idx.append(i)
        
significant_idx = np.array(significant_idx)
print("Selected Features: ", list(significant_idx))
print("There are {} significant features.".format(significant_count))
print()

In [ ]:
X_trainRed = X_train[X_train.columns[list(significant_idx)]]
#X_trainRed.head()
X_testRed = X_test[X_test.columns[list(significant_idx)]]
X_testRed.head()
datafuture = datafuture[datafuture.columns[list(significant_idx)]]

In [ ]:
#Fit and execute the GBC model; append Prediction and Probability to production
# Gradient Boosting
gbc = GradientBoostingClassifier(warm_start=True,verbose=True)
gbc.fit(X_train, y_train)
print("GBC accuracy: ", gbc.score(X_test, y_test))

datafuture['predns'] = gbc.predict(datafuture)
datafuture['probns'] = np.round(gbc.predict_proba(datafuture.drop('predns',axis=1))[:,1],4)
print("ROC accuracy: ", roc_auc_score(y_test,gbc.predict(X_test)))
print("F1 SCore: ", f1_score(y_test,gbc.predict(X_test)))

In [ ]:
datafuture.head()

In [ ]:
# logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
print("LR accuracy: ", lr.score(X_test, y_test))

In [ ]:
%%time
# Random Forest with cross validation
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()
params = [{'n_estimators':[500,600,700],'max_features':[.1,.15,.2,.25,.3,.35],'bootstrap':[True],'warm_start':[True],'verbose':[True],'n_jobs':[8]}]
crf = GridSearchCV(rfc,params,cv=3)
crf.fit(X_train, y_train)
print(crf.best_params_)


print("RFC accuracy: ", crf.score(X_test, y_test))

#datafuture['predns'] = crf.predict(datafuture)
#datafuture['probns'] = np.round(crf.predict_proba(datafuture.drop('predns',axis=1))[:,1],4)
print("ROC accuracy: ", roc_auc_score(y_test,crf.predict(X_test)))
print("F1 SCore: ", f1_score(y_test,crf.predict(X_test)))
joblib.dump(crf,'randomforestclassifier.pkl')
#print(datafuture.shape)
print(datafuture.head())

In [ ]:

from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)



print("RFC accuracy: ", rfc.score(X_test, y_test))

#datafuture['predns'] = crf.predict(datafuture)
#datafuture['probns'] = np.round(crf.predict_proba(datafuture.drop('predns',axis=1))[:,1],4)
print("ROC accuracy: ", roc_auc_score(y_test,crf.predict(X_test)))
print("F1 SCore: ", f1_score(y_test,crf.predict(X_test)))
joblib.dump(crf,'randomforestclassifier.pkl')
#print(datafuture.shape)
print(datafuture.head())

In [ ]:
crf= joblib.load('randomforestclassifier.pkl')
datafuture['predns'] = crf.predict(datafuture)
datafuture['probns'] = np.round(crf.predict_proba(datafuture.drop('predns',axis=1))[:,1],4)
print("ROC accuracy: ", roc_auc_score(y_test,crf.predict(X_test)))
print("F1 SCore: ", f1_score(y_test,crf.predict(X_test)))
joblib.dump(crf,'randomforestclassifier.pkl')
#print(datafuture.shape)
print(datafuture.head())

In [ ]:
print("RFC accuracy: ", crf.score(X_test, y_test))

datafuture['predns'] = crf.predict(datafuture)
datafuture['probns'] = np.round(crf.predict_proba(datafuture.drop('predns',axis=1))[:,1],4)
print("ROC accuracy: ", roc_auc_score(y_test,crf.predict(X_test)))
print("F1 SCore: ", f1_score(y_test,crf.predict(X_test)))

In [ ]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
print("ABC accuracy: ", abc.score(X_test, y_test))
datafuture['predns'] = lr.predict(datafuture)
datafuture['probns'] = np.round(lr.predict_proba(datafuture.drop('predns',axis=1))[:,1],4)
#print(datafuture.shape)
#print(datafuture.head())

In [ ]:
#Join the probability and prediction back to original data on index
results_df = pd.merge(data,datafuture,
                      left_index = True,
                      right_index = True)

In [ ]:
#Narrow down result set to 3 columns to be inserted into database
results_df = results_df[['Contact Serial Number','predns','probns']]

In [ ]:
#Convert CSN from scientific notation to int
results_df['Contact Serial Number'] = results_df['Contact Serial Number'].astype(int)

In [ ]:
#Building metadata table to prep table to be inserted into database, defines table values to be inserted into
meta = MetaData()

predictions = Table(
    'NoShowPredictions', meta,
    Column('ContactSerialNumber', Integer),
    Column('PredictedOutcome', Integer),
    Column('Probability', Float),
)

In [ ]:
coef = lr.coef_ 
Xstd = (X_train.as_matrix(),0)
print(Xstd*coef)
print(X_train.columns.tolist())

In [ ]:
#Connect to the database, insert CSN, Prediction, Probability into oasis_sandbox.dbo.NoShowPredictions

engine = create_engine("(masked)OASIS_Sandbox?driver=SQL+Server+Native+Client+11.0?(masked)")
con= engine.connect() 

results_df = pd.DataFrame({'CSN':results_df['Contact Serial Number'].astype(str),'Pred':results_df['predns'].astype(str),'Probability':results_df['probns'].astype(str)})

for csn, pred, prob in results_df.itertuples(index=False):
     con.execute(predictions.insert().values(ContactSerialNumber=csn,PredictedOutcome=pred, Probability=prob))
